In [3]:
!pip install tensorflow

In [4]:

app = Flask(__name__)

# Cache for storing models and scalers (simple in-memory cache)
model_cache = {}
scaler_cache = {}


def create_model():
    model = Sequential([
        LSTM(100, return_sequences=True, input_shape=(60, 1)),
        LSTM(100),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def get_company_info(ticker):
    url = f"https://query2.finance.yahoo.com/v10/finance/quoteSummary/{ticker}?modules=assetProfile"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else None


def predict_stock(ticker, days=7):
    # Check cache first
    if ticker in model_cache:
        model, scaler = model_cache[ticker], scaler_cache[ticker]
    else:
        model, scaler = create_model(), MinMaxScaler()

    # Download data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * 5)
    df = yf.download(ticker, start=start_date, end=end_date)

    # Prepare data
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

    # Train model
    if ticker not in model_cache:
        model.fit(scaled_data, scaled_data, epochs=10, batch_size=32, verbose=0)
        model_cache[ticker] = model
        scaler_cache[ticker] = scaler

    # Generate predictions
    test_data = scaled_data[-60:]
    predictions = []
    current_batch = test_data.reshape(1, 60, 1)

    for _ in range(days):
        current_pred = model.predict(current_batch, verbose=0)[0]
        predictions.append(current_pred)
        current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

    predictions = scaler.inverse_transform(predictions)
    return {
        'predictions': predictions.flatten().tolist(),
        'history': df['Close'].tolist(),
        'dates': df.index.strftime('%Y-%m-%d').tolist()
    }




ModuleNotFoundError: No module named 'tensorflow.keras.model'